# Cube of MHW_Events

In [10]:
# imports
import numpy as np
import os
from matplotlib import pyplot as plt

from datetime import date

import pandas
import sqlalchemy
import iris

# Load

In [3]:
mhw_file = '/home/xavier/Projects/Oceanography/MHW/db/mhws_allsky_defaults.db'
tst_file = '/home/xavier/Projects/Oceanography/MHW/db/test_mhws_allsky.db'
#
engine = sqlalchemy.create_engine('sqlite:///'+mhw_file)
connection = engine.connect()
connection

In [4]:
mhw_events = pandas.read_sql_table('MHW_Events', con=engine, 
                               columns=['date', 'lon', 'lat', 'duration' , 
                                        'ievent', 'time_peak', 'time_start'])
mhw_events.head()

,date,lon,lat,duration,ievent,time_peak,time_start
0,1982-01-01,0.125,-71.375,13,0,723552,723546
1,1982-12-20,0.125,-71.375,33,1,723901,723899
2,1985-01-30,0.125,-71.375,10,2,724677,724671
3,1988-01-10,0.125,-71.375,9,3,725752,725746
4,1991-01-09,0.125,-71.375,52,4,726858,726841


In [5]:
mhw_events = mhw_events.set_index('date')
mhw_events.head()

,lon,lat,duration,ievent,time_peak,time_start
date,,,,,,
1982-01-01,0.125,-71.375,13,0,723552,723546
1982-12-20,0.125,-71.375,33,1,723901,723899
1985-01-30,0.125,-71.375,10,2,724677,724671
1988-01-10,0.125,-71.375,9,3,725752,725746
1991-01-09,0.125,-71.375,52,4,726858,726841


# Size the Cube

## Load climate for spatial dimensions

In [14]:
climate_file = '/home/xavier/Projects/Oceanography/MHW/db/NOAA_OI_climate_1983-2012.nc'

In [15]:
cube = iris.load(climate_file)
climate = cube[0]
climate

<iris 'Cube' of seasonalT / (C) (day: 366; latitude: 720; longitude: 1440)>

### lat, lon

In [17]:
lat = climate.coord('latitude').points
lat[0:5]

array([-89.875, -89.625, -89.375, -89.125, -88.875], dtype=float32)

In [18]:
lon = climate.coord('longitude').points
lon[0:5]

array([0.125, 0.375, 0.625, 0.875, 1.125], dtype=float32)

## Time

In [19]:
min_time = np.min(mhw_events['time_start'])
min_time

723546

In [20]:
date(1982,1,1).toordinal()

723546

In [22]:
max_time = np.max(mhw_events['time_start'] + mhw_events['duration'])
max_time

737425

In [23]:
date(2019,12,31).toordinal()

737424

In [24]:
max_time-min_time

13879

In [27]:
ntimes = date(2019,12,31).toordinal() - date(1982,1,1).toordinal() + 1
ntimes

13879

# Cube

## Init

In [26]:
cube = np.zeros((720,1440,ntimes), dtype=bool)

## Do It!

In [39]:
ilon = ((mhw_events['lon'].values-0.125)/0.25).astype(np.int32)
ilon

array([   0,    0,    0, ..., 1439, 1439, 1439], dtype=int32)

In [40]:
jlat = ((mhw_events['lat'].values+89.975)/0.25).astype(np.int32)
jlat

array([ 74,  74,  74, ..., 718, 718, 718], dtype=int32)

In [62]:
tstart = mhw_events['time_start'].values
durs = mhw_events['duration'].values
cube[:] = False
for kk in range(len(mhw_events)):
    # Convenience
    #iilon, jjlat, tstart, dur = ilon[kk], jlat[kk], time_start[kk], durations[kk]
    #
    if kk % 1000000 == 0:
        print('kk = {}'.format(kk))
    cube[jlat[kk], ilon[kk], tstart[kk]-min_time:tstart[kk]-min_time+durs[kk]] = True

kk = 0
kk = 1000000
kk = 2000000
kk = 3000000
kk = 4000000
kk = 5000000
kk = 6000000
kk = 7000000
kk = 8000000
kk = 9000000
kk = 10000000
kk = 11000000
kk = 12000000
kk = 13000000
kk = 14000000
kk = 15000000
kk = 16000000
kk = 17000000
kk = 18000000
kk = 19000000
kk = 20000000
kk = 21000000
kk = 22000000
kk = 23000000
kk = 24000000
kk = 25000000
kk = 26000000
kk = 27000000
kk = 28000000
kk = 29000000
kk = 30000000
kk = 31000000
kk = 32000000
kk = 33000000
kk = 34000000
kk = 35000000
kk = 36000000
kk = 37000000
kk = 38000000
kk = 39000000
kk = 40000000
kk = 41000000
kk = 42000000
kk = 43000000
kk = 44000000
kk = 45000000
kk = 46000000
kk = 47000000
kk = 48000000
kk = 49000000
kk = 50000000
kk = 51000000
kk = 52000000
kk = 53000000
kk = 54000000
kk = 55000000
kk = 56000000
kk = 57000000


# Save

In [63]:
np.savez_compressed('/home/xavier/Projects/Oceanography/MHW/db/MHWevent_cube.npz', cube=cube)

# Load

In [64]:
tmp = np.load('/home/xavier/Projects/Oceanography/MHW/db/MHWevent_cube.npz')

In [68]:
tmp2 = tmp['cube']

In [71]:
tmp2.itemsize

1

In [72]:
np.sum(tmp2)

846721274